<a href="https://colab.research.google.com/github/ToluwaseIdowu/My-Analytical-projects/blob/main/DEVELOPMENT_OF_AN_ANDROID_BASED_INEC_SMART_CARD_READER_TRACKING_SYSTEM_USING_A_GLOBAL_POSITIONING_WIRELESS_TECHNOLOGY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import 'package:flutter/material.dart';
import 'package:flutter/services.dart';
import 'MyLocation.dart';
import 'PlatformLevelLocationIssueHandler.dart';
import 'FeatureCollector.dart';
import 'RouteTracker.dart';
import 'SettingsPage.dart';
import 'package:cron/cron.dart';
import 'package:http/http.dart' as http;
import 'package:device_info/device_info.dart';
import 'package:geolocator/geolocator.dart';


class MyAppHome extends StatefulWidget {
  @override
  _MyAppHomeState createState() => _MyAppHomeState();
}

class _MyAppHomeState extends State<MyAppHome> {
  String _placemark = '';
  String place;
  var cron = new Cron();
  DeviceInfoPlugin deviceInfo = DeviceInfoPlugin();
    String devname;
  MyLocation currentLocator = MyLocation(
      null, null, null, null, null, null, null, null, null, null, null, null);
  final String eventChannelName =
      'com.example.itzmeanjan.traceme.locationUpdateEventChannel';
  final String methodChannelName =
      'com.example.itzmeanjan.traceme.locationUpdateMethodChannel';
  bool _areWeGettingLocationUpdates = false;
  PlatformLevelLocationIssueHandler platformLevelLocationIssueHandler;

  @override
  void initState() {
    super.initState();
    platformLevelLocationIssueHandler = PlatformLevelLocationIssueHandler(
        MethodChannel(methodChannelName),
        methodChannelName,
        null,
        eventChannelName);
  }

  @override
  void dispose() {
    // ends location listening service, while disposing app
    if (_areWeGettingLocationUpdates) {
      platformLevelLocationIssueHandler.methodChannel.invokeMethod(
          "stopLocationUpdate"); // calling platform level method, which is defined in MainActivity.kt
    }
    _areWeGettingLocationUpdates = false;
    platformLevelLocationIssueHandler = null;
    super.dispose();
  }

  void _onData(dynamic event) {
    setState(() {
      extractLocationData(event);
    });
  }

  void _onError(dynamic error) {
    setState(() {
      _areWeGettingLocationUpdates = false;
    });
  }

  void extractLocationData(dynamic event) {
    currentLocator.longitude = event['longitude'];
    currentLocator.latitude = event['latitude'];
    currentLocator.time =
        DateTime.fromMillisecondsSinceEpoch(event['time'], isUtc: true);
    currentLocator.altitude = event['altitude'];
    currentLocator.bearing = event['bearing'];
    currentLocator.speed = event['speed'];
    currentLocator.accuracy = event['accuracy'];
    currentLocator.verticalAccuracy = event['verticalAccuracy'];
    currentLocator.bearingAccuracy = event['bearingAccuracy'];
    currentLocator.speedAccuracy = event['speedAccuracy'];
    currentLocator.provider = event['provider'];
    currentLocator.satelliteCount = event['satelliteCount'];
  }
  Future deviceinfo() async {
    AndroidDeviceInfo androidInfo = await deviceInfo.androidInfo;
    devname=androidInfo.model;
    print('Running on ${androidInfo.model}');
  }
  void requestLocationUpdate() {
    final Geolocator _geolocator = Geolocator();
    platformLevelLocationIssueHandler
        .requestLocationPermission()
        .then((bool result) {
      if (result) {
        platformLevelLocationIssueHandler
            .requestToEnableLocation()
            .then((bool resp) async {
          if (resp) {
            platformLevelLocationIssueHandler.eventChannel =
                EventChannel(eventChannelName);
            await platformLevelLocationIssueHandler.methodChannel
                .invokeMethod("fetchLocationDataSourcePreference")
                .then((dynamic id) async {
              // trying to fetch user preferred way for retrieving Device Location. This preference is stored in SharedPreference .
              await platformLevelLocationIssueHandler.methodChannel
                  .invokeMethod("startLocationUpdate", <String, String>{
                "id": id.toString().isNotEmpty ? id.toString() : "1"
              }) // 0 -> google play service based location update request
                  // 1 -> android platform based location update, from android.hardware.gps
                  .then((dynamic value) {
                if (value == 1) {
                  platformLevelLocationIssueHandler.eventChannel
                      .receiveBroadcastStream()
                      .listen(_onData, onError: _onError);
                  setState(() {
                    _areWeGettingLocationUpdates = true;
                  });
                }
              });
            });
          }
        });
      }
    });
    deviceinfo();
    cron.schedule(new Schedule.parse('*/1 * * * *'), () async {
      Position _position = await Geolocator().getCurrentPosition(desiredAccuracy: LocationAccuracy.high);
      _placemark=_position.toString();
      String lon=currentLocator.longitude.toString();
      String lat=currentLocator.latitude.toString();
      String time=currentLocator.time.toIso8601String();
      String speed=currentLocator.speed.toString();
      String alt=currentLocator.altitude.toString();
      String bear=currentLocator.bearingToDirectionName();
      String Info=devname;
      final List<Placemark> placemarks =
      await _geolocator.placemarkFromCoordinates(currentLocator.latitude,currentLocator.longitude);

      if (placemarks != null && placemarks.isNotEmpty) {
        final Placemark pos = placemarks[0];
        setState(() {
          _placemark = pos.thoroughfare + ', ' + pos.locality;
        });
      }
       place=_placemark;
    //  https://idowutoluwase.com/uploaddata/inec.php
     // var uri=Uri.parse("http://bariansystems.com/mobile/inec.php");

         var uri=Uri.parse("http://idowutoluwase.com.ng/mb/inec.php");
         var request=new http.MultipartRequest("POST", uri);
          request.fields['longitude']=lon;
          request.fields['latitude']=lat;
          request.fields['longitude']=lon;
          request.fields['bearing']=bear;
          request.fields['altitude']=alt;
          request.fields['speed']=speed;
          request.fields['address']=place;
          request.fields['deviceinfo']=Info;
          request.fields['devicetime']=time;
      print('Address: $place');
      print('every one minutes $time');
      print('Running latitude: $lat' );
      print('Running lon: $lon' );
      print('Running speed: $speed' );
      print('Running bear: $bear' );
      print('Name: $devname');
      //request.files.add(multipartFile);
      var response=await request.send();
      if(response.statusCode==200){
        print("Insert Successs"+time+lat+lon+speed+bear+devname);


        // _submittedEnquire();
        //await FlutterEmailSender.send(email);
      }else{
        print("upload Failed");

      }

    });

  }

  void stopLocationUpdate() {
    platformLevelLocationIssueHandler.methodChannel
        .invokeMethod("stopLocationUpdate")
        .then((dynamic value) {
      if (value == 1) {
        platformLevelLocationIssueHandler.eventChannel = null;
        setState(() {
          _areWeGettingLocationUpdates = false;
        });
      }
    });
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      drawer: Drawer(
        child: ListView(
          children: <Widget>[
            DrawerHeader(
              decoration: BoxDecoration(
                color: Colors.green,
              ),
              child: Container(
                  child: Align(
                alignment: AlignmentDirectional.center,
                child: Text(
                  "INEC  Tracking System with Wireless Technology",
                  textScaleFactor: 1.5,
                  style: TextStyle(
                    letterSpacing: 4.0,
                    fontSize: 20.0,
                    color: Colors.white,
                  ),
                ),
              )),
            ),
            ListTile(
              title: Text(
                'Feature Collector',
              ),
              leading: Icon(Icons.add),
              onTap: () {
                if (_areWeGettingLocationUpdates) stopLocationUpdate();
                Navigator.push(context, MaterialPageRoute(builder: (context) {
                  return FeatureCollectorHome(
                    location: this.currentLocator,
                    platformLevelLocationIssueHandler:
                        platformLevelLocationIssueHandler,
                  );
                }));
              },
            ),
            ListTile(
              title: Text(
                'Route Tracker',
              ),
              leading: Icon(Icons.directions),
              onTap: () {
                if (_areWeGettingLocationUpdates) stopLocationUpdate();
                Navigator.push(context, MaterialPageRoute(builder: (context) {
                  return RouteTrackerHome(
                    location: this.currentLocator,
                    platformLevelLocationIssueHandler:
                        platformLevelLocationIssueHandler,
                  );
                }));
              },
            ),
            ListTile(
              title: Text(
                'Settings',
              ),
              leading: Icon(Icons.settings),
              onTap: () {
                if (_areWeGettingLocationUpdates) stopLocationUpdate();
                Navigator.push(context, MaterialPageRoute(builder: (context) {
                  return SettingsPage(
                    platformLevelLocationIssueHandler:
                        platformLevelLocationIssueHandler,
                  );
                }));
              },
            ),
          ],
        ),
      ),
      appBar: AppBar(
        title: Text(
          'INEC Card Reader Tracker',
          style: TextStyle(
            color: Colors.white,
          ),
        ),
        elevation: 12,
        backgroundColor: Colors.green,
      ),
      body: Center(
          child: Card(
        elevation: 16.0,
        margin: EdgeInsets.all(12),
        child: Container(
          margin: EdgeInsets.all(4.0),
          padding: EdgeInsets.all(10.0),
          decoration: BoxDecoration(
            borderRadius: BorderRadius.circular(10.0),
            gradient: LinearGradient(
                colors: [Colors.white, Colors.green],
                begin: Alignment.topLeft,
                end: Alignment.bottomRight),
            border: Border.all(
                width: 0.35, style: BorderStyle.solid, color: Colors.white70),
          ),
          child: Column(
            mainAxisSize: MainAxisSize.min,
            children: <Widget>[
              Row(
                mainAxisAlignment: MainAxisAlignment.center,
                children: <Widget>[
                  Text(
                    'Tracker Location Info',
                    textScaleFactor: 1.0,
                    style: TextStyle(
                        fontStyle: FontStyle.italic,
                        letterSpacing: 4,
                        fontWeight: FontWeight.bold,
                        fontSize: 22,
                        shadows: [
                          Shadow(
                            color: Colors.grey,
                            offset: Offset(1.6, 1.6),
                          ),
                        ]),
                  ),
                ],
              ),
              Divider(
                height: 20.0,
                color: Colors.black26,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Address',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.longitude != null
                      ? '$place'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Longitude',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.longitude != null
                      ? '${currentLocator.longitude}'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Latitude',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.latitude != null
                      ? '${currentLocator.latitude}'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Altitude',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.altitude != null
                      ? '${currentLocator.altitude} m'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Bearing',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.bearing != null
                      ? 'along ${currentLocator.bearingToDirectionName()}'
                      : 'NA')
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Speed', style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.speed != null
                      ? '${currentLocator.getSpeedInKiloMetersPerHour()} kmph'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Accuracy',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.accuracy != null
                      ? '${currentLocator.accuracy} m'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Vertical Accuracy',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.verticalAccuracy != null
                      ? '${currentLocator.verticalAccuracy} m'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Bearing Accuracy',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.bearingAccuracy != null
                      ? '${currentLocator.bearingAccuracy}'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Speed Accuracy',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.speedAccuracy != null
                      ? '${currentLocator.getSpeedAccuracyInKiloMetersPerHour()} kmph'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Provider',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.provider != null
                      ? '${currentLocator.provider.toUpperCase()}'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('Satellite Count',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.satelliteCount != null
                      ? '${currentLocator.satelliteCount}'
                      : 'NA'),
                ],
              ),
              Divider(
                height: 10.0,
                color: Colors.white,
              ),
              Row(
                mainAxisAlignment: MainAxisAlignment.spaceBetween,
                children: <Widget>[
                  Text('TimeStamp',
                      style: TextStyle(fontStyle: FontStyle.italic)),
                  VerticalDivider(),
                  Text(currentLocator.time != null
                      ? currentLocator.getParsedTimeString()
                      : 'NA'),
                ],
              ),
              Divider(
                height: 6.0,
                color: Colors.white,
              ),
            ],
          ),
        ),
      )),
      floatingActionButton: FloatingActionButton(
        highlightElevation: 4.0,
        onPressed: !_areWeGettingLocationUpdates
            ? requestLocationUpdate
            : stopLocationUpdate,
        tooltip: !_areWeGettingLocationUpdates
            ? 'Request Location Update'
            : 'Stop Location Update',
        child: !_areWeGettingLocationUpdates
            ? Icon(
                Icons.my_location,
                color: Colors.white,
              )
            : Icon(
                Icons.stop,
                color: Colors.white,
              ),
        backgroundColor:
            !_areWeGettingLocationUpdates ? Colors.green : Colors.red,
      ),
    );
  }
}
